In [7]:
import yfinance as yf
import pandas as pd
from datetime import datetime, timedelta
import csv


# Get the list of NASDAQ companies
nasdaq_tickers = pd.read_csv('../raw_data/nasdaq_tickers_raw.csv')
#nasdaq_tickers= pd.read_csv('https://datahub.io/core/nasdaq-listings/r/nasdaq-listed-symbols.csv')
tickers_total = nasdaq_tickers['Symbol'].tolist()
tickers = tickers_total[0:20]

# Prepare the CSV file
header_written = False
csv_file = 'nasdaq_financial_data.csv'


# Helper function to get the stock price on a specific date
def get_stock_price(ticker, date, stock):
    stock_data = stock.history(start=date, end=date + timedelta(days=7))
    for i in range(7):        
        try:
            adjusted_date = date + timedelta(days=i)
            stock_data_l = stock.history(start=adjusted_date, end=adjusted_date+timedelta(days=1))
            if not stock_data_l.empty:
                return stock_data_l['Close'].iloc[0]
        except Exception as e: 
            print(f"get_stock_price : Error processing ticker {ticker}: {e}")


!
# Function that flattens the data of a dataframe concatenate the columns and put everything in a row
def flatten_data(df, ticker):    
    
    df_u = df.unstack().reset_index()
    df_u['Ticker'] = df_u.apply(lambda row: str(row['level_0'].year) + ' ' + row['level_1'], axis=1)
    df_u.set_index(keys = 'Ticker')
    df_ordered = df_u.drop(columns = ['level_0','level_1'])
    df_transposed = df_ordered.set_index(keys='Ticker').T
    df_transposed['Ticker'] = ticker
    df_transposed.set_index(keys='Ticker',inplace = True)  
    
    return df_transposed


#Compute the stock value ratio (item_stock%/sp500_stock%) in a year
def get_stock_value_ratio(date, ticker, stock, stock_sp):
    try:
        stock_data = get_stock_price(ticker, date, stock)
        if stock_data is not None:
            date_diff = date - timedelta(weeks=52)
            #change in s&p 500 price in %
            sp500_change = get_stock_price("^GSPC", date, stock_sp) / get_stock_price("^GSPC",date_diff, stock_sp)
            #change in stock price in %
            stock_change = get_stock_price(ticker, date, stock)/ get_stock_price(ticker,date_diff, stock)
            #ratio of change_stock / change_s&p500
            ratio = stock_change / sp500_change
            if ratio is not None:
                return ratio
    except Exception as e:
        print(f"get_stock_value_ratio : Error processing ticker {ticker}: {e}")

#function to get the stock value corresponding to the date date

def get_stock_value(df, ticker, stock, stock_sp) :
    lst_stock_value = []
    lst_stock_ratio = []
    for date in df.columns :
        date_str = date.strftime('%Y-%m-%d')
        lst_stock_value.append(get_stock_price(ticker, date, stock))
        lst_stock_ratio.append(get_stock_value_ratio(date, ticker, stock, stock_sp))
        
    df.loc['Stock value'] = lst_stock_value
    df.loc['Ratio ticker sp500'] = lst_stock_ratio
    return df

def create_df(tickers):
    stock_sp = yf.Ticker('^GSPC')
    df_complete = pd.DataFrame()
    for ticker in tickers:  
        df_row = pd.DataFrame()
        try: 
            # Get financial statements for the ticker ticker
            stock = yf.Ticker(ticker)
            balance_sheet = stock.balance_sheet
            income_statement = stock.financials
            cashflow_statement = stock.cashflow
            #stock_history = stock.history
            
            # add the stock value and flatten the dataframe
            #df_bs = get_stock_value(balance_sheet,ticker, stock, stock_sp)
            df_bs_f = flatten_data(balance_sheet,ticker)
            df_is_f = flatten_data(income_statement,ticker)
            df_cs_f = flatten_data(cashflow_statement,ticker)
            #concatenate
            df_row = pd.concat([df_bs_f,df_is_f,df_cs_f],axis=1)
            df_complete = pd.concat([df_complete, df_row])
    
        except Exception as e: 
            print(f"create_df : Error processing ticker {ticker}: {e}")
    return df_complete
        

In [8]:
%%time 
df_ = create_df(tickers)

CPU times: user 2.76 s, sys: 29.1 ms, total: 2.79 s
Wall time: 7.57 s


In [9]:
df_


Ticker,2023 Ordinary Shares Number,2023 Share Issued,2023 Net Debt,2023 Total Debt,2023 Tangible Book Value,2023 Invested Capital,2023 Working Capital,2023 Net Tangible Assets,2023 Common Stock Equity,2023 Total Capitalization,...,2019 Net Investment Properties Purchase And Sale,2019 Sale Of Investment Properties,2019 Purchase Of Investment Properties,2023 Other Equity Interest,2023 Long Term Equity Investment,2023 Investmentsin Joint Venturesat Cost,2022 Investmentsin Joint Venturesat Cost,2021 Investmentsin Joint Venturesat Cost,2019 Dueto Related Parties Current,2019 Investmentsin Joint Venturesat Cost
Ticker,,,,,,,,,,,,,,,,,,,,,
A,292123241.0,292123241.0,1145000000.0,2735000000.0,1410000000.0,8580000000.0,2583000000.0,1410000000.0,5845000000.0,8580000000.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
AA,178472464.0,178472464.0,867000000.0,1915000000.0,4068000000.0,6062000000.0,1375000000.0,4068000000.0,4251000000.0,5983000000.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
AACG,64000260.0,64585618.0,NaN,22606871.0,-142137735.0,113037868.0,-243957610.0,-142137735.0,113037868.0,113037868.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
AACI,8072849.0,8072849.0,2504155.0,2564439.0,16830794.0,19395233.0,-8493234.0,16830794.0,16830794.0,16830794.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
AACIW,8072849.0,8072849.0,2504155.0,2564439.0,16830794.0,19395233.0,-8493234.0,16830794.0,16830794.0,16830794.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
AACT,62500000.0,62500000.0,3094877.0,5000000.0,502674292.0,507674292.0,2135940.0,502674292.0,502674292.0,507674292.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
AADI,24554205.0,24554205.0,NaN,1267000.0,105195000.0,105195000.0,98191000.0,105195000.0,105195000.0,105195000.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
AAGR,57866830.0,57866830.0,2268740.0,11735860.0,-27010851.0,-17526396.0,-26612930.0,-27010851.0,-22583045.0,-22376758.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
AAGRW,57866830.0,57866830.0,2268740.0,11735860.0,-27010851.0,-17526396.0,-26612930.0,-27010851.0,-22583045.0,-22376758.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [7]:
df_['2023 Ratio ticker sp500']

KeyError: '2023 Ratio ticker sp500'